In [1]:
from google.colab import files
import pandas as pd
import numpy as np


In [2]:
# Subir archivos
print("🔼 Sube el archivo de inscritos...")
inscritos = files.upload()
nombre_inscritos = list(inscritos.keys())[0]

🔼 Sube el archivo de inscritos...


Saving Inscritos, admitidos y matriculados al 16-04-2025.xlsx to Inscritos, admitidos y matriculados al 16-04-2025.xlsx


In [12]:
print("🔼 Ahora sube el archivo de metas...")
metas = files.upload()
nombre_metas = list(metas.keys())[0]

🔼 Ahora sube el archivo de metas...


Saving metas.xlsx to metas (2).xlsx


In [14]:
# Cargar los archivos como excel

xls_inscritos = pd.ExcelFile(nombre_inscritos)
xls_metas = pd.ExcelFile(nombre_metas)

# Diccionario de hojas de salida
hojas_final = {}

# Procesar cada hoja
for hoja_inscrito in xls_inscritos.sheet_names:
    print(f"Procesando hoja de inscritos: {hoja_inscrito}")

    # Ajuste para nombre de hoja de metas
    nombre_meta = hoja_inscrito.replace(" 2025-01", "").strip()

    # Leer hojas
    df_base = pd.read_excel(xls_inscritos, sheet_name=hoja_inscrito)
    df_metas = pd.read_excel(xls_metas, sheet_name=nombre_meta)

    # Limpiar columnas
    df_base.columns = df_base.columns.str.strip()
    df_metas.columns = df_metas.columns.str.strip()

    # Renombrar columnas clave
    df_base.rename(columns={
        'Matriculado ': 'Matriculado',
        'Facultad_Desc': 'Facultad_Columna',
        'SMRPRLE1.SMRPRLE_PROGRAM_DESC': 'Carrera_Columna'
    }, inplace=True)

    col_nombre = 'TECNOLOGÍA' if nombre_meta.upper() == 'PUCETEC' else 'CARRERA'
    carrera_key = col_nombre

    # Agrupar inscritos
    agrupado = df_base.groupby(['Facultad_Columna', 'Carrera_Columna']).agg(
        INSCRITOS=('Carrera_Columna', 'count'),
        ADMITIDOS=('Aceptado_por_institucion', lambda x: (x == 35).sum()),
        MATRICULADOS=('Matriculado', lambda x: (x == 1).sum()),
        RECAUDACIÓN=('Recaudación', 'sum')
    ).reset_index()

    agrupado.rename(columns={
        'Facultad_Columna': 'DOMINIO',
        'Carrera_Columna': carrera_key
    }, inplace=True)

    # Lógica específica para GRADO
    if nombre_meta.upper() == 'GRADO':
        df_metas = df_metas.drop_duplicates(subset=['DOMINIO', 'CARRERA'])

        df_full = pd.merge(
            agrupado,
            df_metas,
            how='left',
            left_on=['DOMINIO', carrera_key],
            right_on=['DOMINIO', 'CARRERA']
        )

        if 'CARRERA' in df_full.columns and carrera_key != 'CARRERA':
            df_full.drop(columns=['CARRERA'], inplace=True)

        # Calcular META DOMINIO y % CUMPLIMIENTO POR DOMINIO
        meta_dominio = df_full.groupby('DOMINIO')['META'].sum().reset_index()
        df_full = pd.merge(df_full, meta_dominio, on='DOMINIO', suffixes=('', '_DOMINIO'))
        df_full.rename(columns={'META_DOMINIO': 'META DOMINIO'}, inplace=True)

        df_full['% CUMPLIMIENTO POR DOMINIO'] = np.where(
            df_full['META DOMINIO'] > 0,
            (df_full['INSCRITOS'] / df_full['META DOMINIO'] * 100).round(0).astype(str) + '%',
            '0%'
        )

    else:
        df_metas = df_metas.drop_duplicates(subset=[carrera_key])
        df_full = pd.merge(agrupado, df_metas, how='left', on=carrera_key)

    # Calcular métricas comunes
    df_full['PROMEDIO PAGO'] = np.where(df_full['MATRICULADOS'] > 0,
                                        df_full['RECAUDACIÓN'] / df_full['MATRICULADOS'], 0)
    df_full['% CUMPLIMIENTO MATRICULADOS'] = np.where(df_full['META'] > 0,
                                                      (df_full['MATRICULADOS'] / df_full['META'] * 100).round(0).astype(str) + '%',
                                                      '0%')
    df_full['% CUMPLIMIENTO PRESUPUESTO'] = np.where(df_full['META PRESUPUESTO'] > 0,
                                                     (df_full['RECAUDACIÓN'] / df_full['META PRESUPUESTO'] * 100).round(0).astype(str) + '%',
                                                     '0%')

    # Ordenar columnas
    if nombre_meta.upper() == 'PUCETEC':
        columnas_finales = [
            carrera_key, 'META', 'INSCRITOS', 'ADMITIDOS', 'MATRICULADOS',
            'RECAUDACIÓN', '% CUMPLIMIENTO MATRICULADOS',
            'META PRESUPUESTO', '% CUMPLIMIENTO PRESUPUESTO'
        ]
    elif nombre_meta.upper() == 'GRADO':
        columnas_finales = [
            'DOMINIO', 'META DOMINIO', '% CUMPLIMIENTO POR DOMINIO',
            carrera_key, 'META', 'INSCRITOS', 'ADMITIDOS',
            'MATRICULADOS', 'RECAUDACIÓN', 'PROMEDIO PAGO',
            '% CUMPLIMIENTO MATRICULADOS', 'META PRESUPUESTO', '% CUMPLIMIENTO PRESUPUESTO'
        ]
    else:
        columnas_finales = [
            'DOMINIO', carrera_key, 'META', 'INSCRITOS', 'ADMITIDOS',
            'MATRICULADOS', 'RECAUDACIÓN', 'PROMEDIO PAGO',
            '% CUMPLIMIENTO MATRICULADOS', 'META PRESUPUESTO', '% CUMPLIMIENTO PRESUPUESTO'
        ]

    df_final = df_full[columnas_finales]
    hojas_final[hoja_inscrito] = df_final

# Guardar Excel final
nombre_final = "REPORTE_MATRICULADOS_FINAL_DOMINIO.xlsx"
with pd.ExcelWriter(nombre_final, engine='openpyxl') as writer:
    for hoja, df in hojas_final.items():
        df.to_excel(writer, sheet_name=hoja, index=False)

files.download(nombre_final)


Procesando hoja de inscritos: GRADO 2025-01
Procesando hoja de inscritos: PUCETEC 2025-01
Procesando hoja de inscritos: POSGRADO HÍBRIDO 2025-01
Procesando hoja de inscritos: POSGRADO VIRTUAL 2025-01


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>